In [9]:
from pathlib import Path

import numpy as np
from moviepy.editor import AudioFileClip, VideoClip
from IPython.display import Video

ROOT = Path("..")

In [6]:
# ---------- CONFIG ----------
AUDIO_PATH = str(ROOT / "data/sample.mp3")
OUTPUT_PATH = str(ROOT / "output/sample.mp4")
WIDTH, HEIGHT = 1280, 720
FPS = 30
BACKGROUND_COLOR = (0, 0, 0)       # black
WAVE_COLOR = (255, 255, 255)       # white
CURSOR_COLOR = (0, 255, 0)         # green
CURSOR_WIDTH = 4
# -----------------------------

In [10]:

# Load the audio
audio_clip = AudioFileClip(AUDIO_PATH)
duration = audio_clip.duration

# Convert audio to a numpy array (mono)
# fps here is only for analysis, not video fps
analysis_fps = 44100

# Manually collect chunks into a list, then concatenate
chunks = list(
    audio_clip.iter_chunks(
        fps=analysis_fps,
        quantize=True,  # get int16-ish PCM
        nbytes=2,
        chunksize=1024
    )
)

audio_array = np.concatenate(chunks, axis=0).astype(np.float32)

# Convert to mono if stereo
if audio_array.ndim == 2:
    mono = audio_array.mean(axis=1)
else:
    mono = audio_array

# Normalize to [-1, 1]
mono = mono / np.max(np.abs(mono))

# Precompute a waveform of width = video WIDTH
# Each x-pixel corresponds to a segment of the audio
indices = np.linspace(0, len(mono) - 1, WIDTH).astype(int)
wave = mono[indices]

# Vertical scaling
center_y = HEIGHT // 2
amplitude_px = int(HEIGHT * 0.4)  # waveform height

# Precompute a static waveform image (without cursor)
base_frame = np.full((HEIGHT, WIDTH, 3), BACKGROUND_COLOR, dtype=np.uint8)

for x in range(WIDTH):
    val = wave[x]
    y0 = int(center_y - val * amplitude_px)
    y1 = int(center_y + val * amplitude_px)

    # Clip to bounds
    y0 = max(0, min(HEIGHT - 1, y0))
    y1 = max(0, min(HEIGHT - 1, y1))

    if y0 > y1:
        y0, y1 = y1, y0

    base_frame[y0:y1 + 1, x, :] = WAVE_COLOR

# Function to generate each video frame
def make_frame(t):
    # Copy the static waveform
    frame = base_frame.copy()

    # Cursor position based on time
    progress = t / duration  # 0 -> 1
    progress = max(0, min(1, progress))
    cursor_x = int(progress * (WIDTH - 1))

    x_start = max(0, cursor_x - CURSOR_WIDTH // 2)
    x_end = min(WIDTH, cursor_x + CURSOR_WIDTH // 2 + 1)

    frame[:, x_start:x_end, :] = CURSOR_COLOR

    return frame

# Create video clip
video_clip = VideoClip(make_frame, duration=duration)
video_clip = video_clip.set_audio(audio_clip)

# Export
video_clip.write_videofile(
    OUTPUT_PATH,
    fps=FPS,
    codec="libx264",
    audio_codec="aac"
)

Video(url=OUTPUT_PATH)

Moviepy - Building video ../output/sample.mp4.
MoviePy - Writing audio in sampleTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ../output/sample.mp4



Moviepy - Done !
Moviepy - video ready ../output/sample.mp4
